In [70]:
import numpy as np
import random as rd
import math
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log

In [71]:
class Dtrees:
    def __init__(self,k,m):
        self.k = k    #number of features
        self.m = m    #number of data points
        self.arrofarrX = []
        self.arrY = []
        
        for i in range(self.m):
            new = []
            tmpX = self.generateX()
            tmpW = self.generateW()
            tmpY = self.computeY(tmpX,tmpW)
            self.arrofarrX.append(tmpX)
            self.arrY.append(tmpY)
    
    def ran(self,x,y):
        if rd.uniform(x,y) > 0.5:
            return 1
        else:
            return 0
        
    def ran2(self,x,y):
        if rd.uniform(x,y) > 0.25:
            return 1
        else:
            return 0
        
    def generateX(self):
        new = []
        if self.ran(0,1) == 1:
            new.append(1)
        else:
            new.append(0)
        
        for x in range (1,self.k):
            if self.ran2(0,1) == 1:
                new.append(new[x-1])
            else:
                new.append(1-new[x-1])
        return new
    
    def generateW(self):
        new_w = []
        for x in range (1,self.k):
            de = 0
            for y in range(1,x+1):
                de = de+ math.pow(0.9,y)
            res = math.pow(0.9,x) / de
            new_w.append(res)
        return new_w
    
    def computeY(self,arrX,arrW):
        res = 0
        for x in range (1,self.k):
            res+= arrX[x]*arrW[x-1]
#         print("current result is %d",res)
        if res >= 0.5:
            return arrX[0]
        else:
            return 1 - arrX[0]
        

# Transfer data into data frame

In [196]:
def insertDS(ds,arr):
    tmp = len(arr)
    for x in range(tmp):
        ds[x] = arr[x]
    return ds

# Above

In [74]:
dataset = {'Taste':['Salty','Spicy','Spicy','Spicy','Spicy','Sweet','Salty','Sweet','Spicy','Salty'],
       'Temperature':['Hot','Hot','Hot','Cold','Hot','Cold','Cold','Hot','Cold','Hot'],
       'Texture':['Soft','Soft','Hard','Hard','Hard','Soft','Soft','Soft','Soft','Hard'],
       'Eat':['No','No','Yes','No','Yes','Yes','No','Yes','Yes','Yes']}

In [75]:

df = pd.DataFrame(dataset,columns=['Taste','Temperature','Texture','Eat'])

In [76]:
entropy_node = 0  #Initialize Entropy
values = df.Eat.unique()  #Unique objects - 'Yes', 'No'
for value in values:
    fraction = df.Eat.value_counts()[value]/len(df.Eat)  
    entropy_node += -fraction*np.log2(fraction)

In [77]:
df

,Taste,Temperature,Texture,Eat
0,Salty,Hot,Soft,No
1,Spicy,Hot,Soft,No
2,Spicy,Hot,Hard,Yes
3,Spicy,Cold,Hard,No
4,Spicy,Hot,Hard,Yes
5,Sweet,Cold,Soft,Yes
6,Salty,Cold,Soft,No
7,Sweet,Hot,Soft,Yes
8,Spicy,Cold,Soft,Yes
9,Salty,Hot,Hard,Yes


In [78]:
len(values)

2

In [79]:
df.Eat.value_counts()

Yes    6
No     4
Name: Eat, dtype: int64

In [80]:
df.Eat.value_counts()[0]

6

In [81]:
df.Eat.unique()

array(['No', 'Yes'], dtype=object)

In [82]:
attribute = 'Taste'
target_variables = df.Eat.unique()  #This gives all 'Yes' and 'No'
variables = df[attribute].unique()    #This gives different features in that attribute (like 'Sweet')
entropy_attribute = 0
for variable in variables:
    entropy_each_feature = 0
    for target_variable in target_variables:
        num = len(df[attribute][df[attribute]==variable][df.Eat ==target_variable]) #numerator
        den = len(df[attribute][df[attribute]==variable])  #denominator
        fraction = num/(den+eps)  #pi
        entropy_each_feature += -fraction*log(fraction+eps) #This calculates entropy for one feature like 'Sweet'
    fraction2 = den/len(df)
    
    
    entropy_attribute += -fraction2*entropy_each_feature   #Sums up all the entropy ETaste

# START BUILDING DECISION TREE

# Compute $H(Y)$

In [289]:
d = Dtrees(15,5)
df = pd.DataFrame(d.arrofarrX)
L = len(d.arrofarrX[0])


In [290]:
L

15

In [279]:
d.arrofarrX

[[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1],
 [1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0]]

In [280]:
d.arrY

[1, 0, 0, 1, 1]

In [281]:
df = pd.DataFrame(d.arrofarrX)

In [282]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1
1,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0
2,0,1,0,0,0,0,0,0,1,1,1,1,1,0,0
3,0,0,0,0,0,0,0,0,1,0,1,1,1,0,1
4,1,1,1,1,0,0,0,1,1,0,0,0,1,0,0


In [283]:
df[15] = d.arrY

In [284]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1
1,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0
2,0,1,0,0,0,0,0,0,1,1,1,1,1,0,0,0
3,0,0,0,0,0,0,0,0,1,0,1,1,1,0,1,1
4,1,1,1,1,0,0,0,1,1,0,0,0,1,0,0,1


In [237]:
def find_entropy(df):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy

# Compute $H(Y|X) = \sum_{x}P(X = x)[-\sum_{y}P(Y=y|X=x)\log P(Y=y|X=x)]$

In [238]:
def find_entropy_attribute(df,attribute):
  Class = df.keys()[-1]   #To make the code generic, changing target variable class name
  target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
  variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
  entropy2 = 0
  for variable in variables:
      entropy = 0
      for target_variable in target_variables:
          num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
          den = len(df[attribute][df[attribute]==variable])
          fraction = num/(den+eps)
          entropy += -fraction*log(fraction+eps)
      fraction2 = den/len(df)
      entropy2 += -fraction2*entropy
  return abs(entropy2)

# Find the the attribute that could lead to the biggest entropy

In [239]:
def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
#         Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    print(IG)
    return df.keys()[:-1][np.argmax(IG)]

In [240]:
ans = find_winner(arr)

[6.661338147750939e-16, 0.39581560200335886, 6.661338147750939e-16, 0.1245112497836538, 6.661338147750939e-16, 0.609986547010988, 0.609986547010988, 0.03485155455967781, 6.661338147750939e-16, 6.661338147750939e-16]


# get subtable since we find a attribute to cut the table

In [242]:
def get_subtable(df, node,value):
  return df[df[node] == value].reset_index(drop=True)

In [253]:
ans = arr[arr[0]==0].reset_index(drop=True)